# Generic NN trainer

In [1]:
import sys

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")


from dictionary import Dictionary
from lstm import lstm
from predictor.ticks import Ticks
from utils.utils import print_bin_predictions_match
from utils.plot_utils import plot_marks

## Setup the generic parameters for the trainer

In [2]:
argv = [
    "",
    "-c",
    "../params/params.eurusd.yaml",
    "-f",
    "../data/EURUSD/EURUSD_2018_2020.csv",
    "--window",
    "14",
    "--epochs",
    "100",
    "train",
]
params = Dictionary(args=argv)


def read_ticks():
    global params

    ticks = Ticks(params, csv_file=params.input_file).scale()
    ticks.append_indicator(["trend", "median_filter", "change"])
    ticks.append_indicator("gmf", monotonic_window=7, mf_window=3, sigma=5)
    ticks.append_indicator("gmf_mono", monotonic_window=7, mf_window=3, sigma=5)
    ticks.append_indicator("trend", column_name="gmf")
    return ticks


ticks = read_ticks()
ticks.data.head()

2020-10-21 12:50:18 - INFO  - Dictionary:__init__             - Using configuration parameters from: ../params/params.eurusd.yaml


,open,high,low,close,close_trend,med_filter,change,returns,gmf,gmf_mono,gmf_trend
Datetime,,,,,,,,,,,
2018-01-01,1.2274,1.1598,1.2794,1.2442,1.0,1.2423,0.0000,0.000000,0.001247,0.0,1.0
2018-01-02,1.2292,1.2731,1.2922,1.3358,1.0,1.2442,0.0916,0.073622,0.001905,0.0,1.0
2018-01-03,1.3218,1.2430,1.2794,1.2423,0.0,1.2753,-0.0935,-0.069996,0.003268,0.0,1.0
2018-01-04,1.2292,1.2873,1.2849,1.3449,1.0,1.2918,0.1026,0.082589,0.005403,0.0,1.0
2018-01-05,1.3309,1.2767,1.3160,1.2753,0.0,1.2753,-0.0696,-0.051751,0.008347,0.0,1.0


In [3]:
# plot_marks(data=ticks.data, signal="close", marks="gmf_mono", dark=True)
#
# plot_marks(data=ticks.data, signal="gmf", marks="gmf_mono", dark=True)

# Train a network to predict CLOSE with GMF, MONOTONIC SIGN and GMF TREND

In [4]:
ticks = read_ticks()
params.epochs = 150

X_train_close, y_train_close, X_test_close, y_test_close = ticks.prepare_for_training(
    predict="close_trend", train_columns=["gmf", "gmf_mono", "gmf_trend"]
)

In [26]:
nn1 = lstm(params).build()
nn1.start_training(X_train_close, y_train_close, name=None)
yhat_close_trend, acc = nn1.evaluate(X_test_close, y_test_close)

2020-10-20 19:10:54 - INFO  - lstm:build                      - NN lstm_1layers created
2020-10-20 19:10:54 - INFO  - lstm:start_training             - Training for 150 epochs...
Epoch 150/150 - Acc:0.67 (↑0.71/↓0.57) - Val:0.62 (↑0.72/↓0.55) | [====================]
3/3 [==============================] - 0s 1ms/step - loss: 0.7871 - accuracy: 0.6310
2020-10-20 19:11:29 - INFO  - lstm:evaluate                   - Predictions (yhat): (84, 1)
2020-10-20 19:11:29 - INFO  - lstm:evaluate                   - Accuracy: 0.63
2020-10-20 19:11:29 - INFO  - lstm:evaluate                   - Trend acc.: 0.63


In [28]:
print(nn1)
print_bin_predictions_match(y_test_close, yhat_close_trend)

LSTMb(1l;28u;d=0.20;lr=0.01;[W=14;E=150;BS=8])

00 | 0 | 0.47 | 0 | 0.44 | 0 | 0.40 | 1 | 0.05 | 1 | 0.18 | 1 | 0.18 | 0 | 0.94 | 0 | 0.43 | 0 | 0.37 | 
09 | 1 | 0.55 | 1 | 0.72 | 0 | 0.80 | 1 | 0.70 | 0 | 0.57 | 1 | 0.46 | 1 | 0.66 | 1 | 0.76 | 0 | 0.73 | 
18 | 1 | 0.66 | 0 | 0.62 | 1 | 0.62 | 1 | 0.65 | 1 | 0.67 | 1 | 0.68 | 1 | 0.69 | 0 | 0.70 | 1 | 0.70 | 
27 | 1 | 0.71 | 1 | 0.72 | 1 | 0.73 | 1 | 0.74 | 1 | 0.75 | 1 | 0.75 | 0 | 0.76 | 1 | 0.76 | 0 | 0.76 | 
36 | 1 | 0.82 | 1 | 0.86 | 0 | 0.95 | 1 | 0.97 | 0 | 0.97 | 1 | 0.97 | 1 | 0.91 | 1 | 0.53 | 0 | 0.58 | 
45 | 0 | 0.92 | 0 | 0.88 | 0 | 0.84 | 1 | 0.74 | 1 | 0.81 | 1 | 0.87 | 1 | 0.85 | 1 | 0.81 | 1 | 0.81 | 
54 | 0 | 0.79 | 1 | 0.78 | 0 | 0.76 | 1 | 0.74 | 0 | 0.72 | 1 | 0.71 | 1 | 0.70 | 0 | 0.70 | 1 | 0.69 | 
63 | 1 | 0.69 | 1 | 0.69 | 0 | 0.68 | 0 | 0.66 | 0 | 0.65 | 0 | 0.63 | 0 | 0.62 | 0 | 0.60 | 0 | 0.59 | 
72 | 1 | 0.58 | 1 | 0.55 | 1 | 0.50 | 0 | 0.45 | 1 | 0.41 | 0 | 0.37 | 0 | 0.33 | 1 | 0.30 | 0 | 0.26 | 
81 | 0 

# Save the model and the scaler!

In [29]:
params.models_dir = "../staging/EURUSD/"
nn1.save()

2020-10-20 19:11:50 - INFO  - lstm:save                       - Saved model and weights (../staging/EURUSD/EURUSD_2018_2020_w14_e150)


In [4]:
print(nn1)

NameError: name 'nn1' is not defined

In [5]:
from os.path import splitext, basename

dataset_name = splitext(basename(params.input_file))[0]
ticks.save_scaler()

RobustScaler saved at: ../staging/EURUSD/scaler_EURUSD_2018_2020.bin


## Load it all!

In [10]:
kk = lstm(params).load("EURUSD_2018_2020_w28_e160")

yhat_close_trend, acc = kk.evaluate(X_test_close, y_test_close)
print_bin_predictions_match(y_test_close, yhat_close_trend)

[Determinism: Random seeds reset]
3/3 [==============================] - 0s 2ms/step - loss: 0.7902 - accuracy: 0.5542
2020-10-15 10:05:08 - INFO  - lstm:evaluate                   - Predictions (yhat): (83, 1)
2020-10-15 10:05:08 - INFO  - lstm:evaluate                   - Accuracy: 0.55
2020-10-15 10:05:08 - INFO  - lstm:evaluate                   - Trend acc.: 0.55

00 | 0 | 0.44 | 0 | 0.65 | 1 | 0.43 | 1 | 0.14 | 1 | 0.49 | 0 | 0.92 | 0 | 0.93 | 0 | 0.64 | 1 | 0.33 | 
09 | 1 | 0.45 | 0 | 0.83 | 1 | 0.62 | 0 | 0.42 | 1 | 0.51 | 1 | 0.70 | 1 | 0.81 | 0 | 0.76 | 1 | 0.67 | 
18 | 0 | 0.65 | 1 | 0.69 | 1 | 0.73 | 1 | 0.75 | 1 | 0.74 | 1 | 0.74 | 0 | 0.75 | 1 | 0.76 | 1 | 0.78 | 
27 | 1 | 0.79 | 1 | 0.80 | 1 | 0.81 | 1 | 0.82 | 1 | 0.83 | 0 | 0.84 | 1 | 0.85 | 0 | 0.85 | 1 | 0.80 | 
36 | 1 | 0.80 | 0 | 0.86 | 1 | 0.90 | 0 | 0.90 | 1 | 0.85 | 1 | 0.32 | 1 | 0.47 | 0 | 0.63 | 0 | 0.05 | 
45 | 0 | 0.15 | 0 | 0.77 | 1 | 0.62 | 1 | 0.03 | 1 | 0.76 | 1 | 0.79 | 1 | 0.91 | 1 | 0.90 | 0 | 0.59 |

In [11]:
print(kk)

LSTMb(1l;28u;d=0.20;lr=0.01;[W=28;E=160;BS=16])


## Make prediction with the loaded model

In [8]:
yhat_close_trend, acc_close_trend = nn1.evaluate(X_train_close, y_train_close)

24/24 [==============================] - 0s 2ms/step - loss: 0.5680 - accuracy: 0.6945
2020-10-13 20:26:53 - INFO  - lstm:evaluate                   - Predictions (yhat): (743, 1)
2020-10-13 20:26:53 - INFO  - lstm:evaluate                   - Accuracy: 0.69
2020-10-13 20:26:53 - INFO  - lstm:evaluate                   - Trend acc.: 0.69


----

# Train another network to predict GMF TREND with GMF and MONOTONIC SIGN

In [128]:
ticks = read_ticks()
params.epochs = 100
X_train_gmf, y_train_gmf, X_test_gmf, y_test_gmf = ticks.prepare_for_training(
    predict="gmf_trend", train_columns=["gmf", "gmf_mono"]
)

nn2 = lstm(params, binary=True)
nn2.start_training(X_train_gmf, y_train_gmf, name=None)
yhat, acc = nn2.evaluate(X_test_gmf, y_test_gmf)

print(nn2)
print_bin_predictions_match(y_test_gmf, yhat_gmf_trend)

[Determinism: Random seeds reset]
[Determinism: Random seeds reset]
2020-10-07 19:39:51 - INFO  - lstm:_build_model               - NN lstm_1layers created
2020-10-07 19:39:51 - INFO  - lstm:start_training             - Training for 100 epochs...
Epoch 100/100 - Acc:0.91 (↑0.92/↓0.69) - Val:0.75 (↑0.89/↓0.63) | [====================]
3/3 [==============================] - 0s 2ms/step - loss: 0.2732 - accuracy: 0.9405
2020-10-07 19:40:38 - INFO  - lstm:evaluate                   - Predictions (yhat): (84, 1)
2020-10-07 19:40:38 - INFO  - lstm:evaluate                   - Accuracy: 0.94
2020-10-07 19:40:38 - INFO  - lstm:evaluate                   - Trend acc.: 0.94
LSTMb(1l. 42u. d=0.10 lr=0.01 [W=14 E=100 BS=8])

00 | 0 | 0.06 | 0 | 0.14 | 1 | 0.18 | 1 | 0.99 | 1 | 1.00 | 1 | 1.00 | 1 | 1.00 | 1 | 0.70 | 1 | 0.69 | 
09 | 1 | 0.87 | 1 | 0.90 | 1 | 0.55 | 1 | 0.89 | 1 | 0.62 | 1 | 0.84 | 1 | 0.71 | 1 | 0.85 | 1 | 0.64 | 
18 | 1 | 0.74 | 1 | 0.64 | 1 | 0.64 | 1 | 0.62 | 1 | 0.62 | 1 | 0.6

In [134]:
yhat_gmf_trend, acc_gmf_trend = nn2.evaluate(X_train_gmf, y_train_gmf)

24/24 [==============================] - 0s 2ms/step - loss: 0.2890 - accuracy: 0.9101
2020-10-07 19:42:18 - INFO  - lstm:evaluate                   - Predictions (yhat): (756, 1)
2020-10-07 19:42:18 - INFO  - lstm:evaluate                   - Accuracy: 0.91
2020-10-07 19:42:18 - INFO  - lstm:evaluate                   - Trend acc.: 0.91


# Ensemble of the output from the two networkss

In [136]:
import pandas as pd

ensemble = pd.DataFrame(
    {
        "yhat_close": yhat_close_trend.reshape(-1,),
        "yhat_gmf": yhat_gmf_trend.reshape(-1,),
        "y_close": y_train_close.reshape(-1,),
        "y_gmf": y_train_gmf.reshape(-1,),
    }
)
ensemble.head()

,yhat_close,yhat_gmf,y_close,y_gmf
0,0.816025,0.617519,0.0,1.0
1,0.814545,0.629542,1.0,1.0
2,0.783705,0.604789,0.0,0.0
3,0.846610,0.025967,1.0,0.0
4,0.874096,0.001961,1.0,0.0


Entrenar la SVM con los datos usados para el entrenamiento de las LSTM, pero hacer la validación con los datos de test.

In [149]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

svclassifier = SVC(kernel="rbf")
svclassifier.fit(X, y)

yhat_close_test, _ = nn1.evaluate(X_test_close, y_test_gmf)
yhat_gmf_test, _ = nn2.evaluate(X_test_gmf, y_test_gmf)

X_test = pd.DataFrame(
    {
        "yhat_close": yhat_close_test.reshape(-1,),
        "yhat_gmf": yhat_gmf_test.reshape(-1,),
    }
)
y_test = y_test_close.reshape(-1,)

3/3 [==============================] - 0s 2ms/step - loss: 0.9058 - accuracy: 0.5357
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Predictions (yhat): (84, 1)
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Accuracy: 0.54
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Trend acc.: 0.55
3/3 [==============================] - 0s 1ms/step - loss: 0.2732 - accuracy: 0.9405
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Predictions (yhat): (84, 1)
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Accuracy: 0.94
2020-10-07 19:51:13 - INFO  - lstm:evaluate                   - Trend acc.: 0.94


In [150]:
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[16 21]
 [14 33]]
              precision    recall  f1-score   support

         0.0       0.53      0.43      0.48        37
         1.0       0.61      0.70      0.65        47

    accuracy                           0.58        84
   macro avg       0.57      0.57      0.57        84
weighted avg       0.58      0.58      0.58        84

